In [49]:
import numpy as np
import matplotlib.pyplot as plt
from lmfit import Model, fit_report, Minimizer, Parameters, Parameter

print('Matlab ref omega: ', 1.752690409810814)
# Define the function to be fitted
def func(x, a, b, omega, phi, c):
    return a * np.exp(-b * x) * np.sin(omega * x + phi) + c

# Load data from text files
xdata = np.loadtxt('xdata.txt')
ydata = np.loadtxt('data150_6.txt')

# Define the model to be fitted
model = Model(func)

# Set initial parameter values and bounds
#params = model.make_params(a=0.1, b=0.4, omega=1.0, phi=0.1, c=1.0)
params = model.make_params(a=0.1, b=0.8234, omega=2.5, phi=1, c=1.5)
# 0.0971317812358475 0.823457828327293 2.5 1 1.5
params['a'].min = 0.0
params['a'].max = 1.0
params['b'].min = 0.0
#params['b'].max = 5.0
params['omega'].min = 0.0
params['phi'].min = 0
params['phi'].max = 2*np.pi

# Define the options for each algorithm
leastsq_opts = {
    'max_nfev': 1000,
}

least_squares_opts = {
    'max_nfev': 1000,
}

differential_evolution_opts = {
    'popsize': 15,
    'tol': 1e-8,
    'strategy': 'best1bin',
    'mutation': 1.5,
    'recombination': 0.7,
    'maxiter': 500,
    'disp': True,
}

brute_opts = {
    'ranges': [(0.0, 1.0), (0.0, 1.0), (0.0, 1.0), (0.0, 2*np.pi), (0.0, 1.0)],
    'brute_step': [0.1, 0.1, 0.1, 0.1, 0.1],
    'finish': None, # Disable the local minimization step
    'workers': -1 # Use all available cores for parallelization
}


basinhopping_opts = {

}

# Fit the model to the data with different algorithms and track the progress
#methods = ['leastsq', 'least_squares', 'differential_evolution', 'brute', 'basinhopping']
methods = ['leastsq', 'least_squares', 'basinhopping']

results = {}
for method in methods:
    if method == 'leastsq':
        result = model.fit(ydata, params, x=xdata, method=method, **leastsq_opts)
    elif method == 'least_squares':
        result = model.fit(ydata, params, x=xdata, method=method, **least_squares_opts)
    elif method == 'differential_evolution':
        result = model.fit(ydata, params, x=xdata, method=method, **differential_evolution_opts)
    elif method == 'brute':
        result = model.fit(ydata, params, x=xdata, method=method, **brute_opts)
    elif method == 'basinhopping':
        result = model.fit(ydata, params, x=xdata, method=method, **basinhopping_opts)
    results[method] = result
    print(f"FINAL RESULT ({method}): {fit_report(result)}")

Matlab ref omega:  1.752690409810814
FINAL RESULT (leastsq): [[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 81
    # data points      = 6000
    # variables        = 5
    chi-square         = 0.00732161
    reduced chi-square = 1.2213e-06
    Akaike info crit   = -81688.6380
    Bayesian info crit = -81655.1404
    R-squared          = 0.99430520
[[Variables]]
    a:      0.09263004 +/- 1.3195e-04 (0.14%) (init = 0.1)
    b:      0.18906723 +/- 3.6961e-04 (0.20%) (init = 0.8234)
    omega:  1.75745237 +/- 3.6387e-04 (0.02%) (init = 2.5)
    phi:    1.82342064 +/- 0.00135037 (0.07%) (init = 1)
    c:      2.59106292 +/- 1.4335e-05 (0.00%) (init = 1.5)
[[Correlations]] (unreported correlations are < 0.100)
    C(a, b)       = +0.7207
    C(omega, phi) = -0.7099
    C(a, phi)     = +0.2051
    C(b, phi)     = +0.1485
    C(a, omega)   = -0.1471
    C(b, omega)   = -0.1060
FINAL RESULT (least_squares): [[Fit Statistics]]
    # fitting method   = least_squares

In [50]:
import numpy as np
from lmfit import Model, fit_report, minimize

print('Matlab ref omega: ', 1.752690409810814)

# Define the function to be fitted
def func(x, a, b, omega, phi, c):
    return a * np.exp(-b * x) * np.sin(omega * x + phi) + c

# Define the residual function
def residual(params, x, y):
    return y - func(x, **params)

# Load data from text files
xdata = np.loadtxt('xdata.txt')
ydata = np.loadtxt('data150_6.txt')

# Mask 20% of the data
n_mask = int(len(ydata) * 0.0)
mask_indices = np.random.choice(len(ydata), size=n_mask, replace=False)
ydata[mask_indices] = np.nan
# Define the boolean mask
mask = np.logical_not(np.isnan(ydata))
# Apply the mask to the data
xdata = xdata[mask]
ydata = ydata[mask]


# Define the model to be fitted
model = Model(func)

# Set initial parameter values and bounds
params = model.make_params(a=1.0, b=1.0, omega=1.0, phi=1.0, c=1.0)


params['a'].min = 0.0
params['b'].min = 0.0
params['omega'].min = 0.0
params['phi'].min = -np.pi
params['phi'].max = np.pi
params['a'].max = 5.0
params['b'].max = 5.0
params['omega'].max = 5.0
params['c'].min=0.0
params['c'].max=5.0



# Fit the model to the data using different methods
result_leastsq = model.fit(ydata, params, x=xdata, method='leastsq')
result_least_squares_bounds = model.fit(ydata, params, x=xdata, method='least_squares')
result_nelder = minimize(residual, params, args=(xdata, ydata), method='nelder')
result_bfgs = minimize(residual, params, args=(xdata, ydata), method='bfgs')
result_differential_evolution = minimize(residual, params, args=(xdata, ydata), method= 'differential_evolution')
result_basinhopping = minimize(residual, params, args=(xdata, ydata), method= 'basinhopping')
#result_brute = minimize(residual, params, args=(xdata, ydata), method= 'brute')


# Print the fit results for each method
print("Results using least sq:")
print(fit_report(result_leastsq))
print("\nResults using least squares with bounds:")
print(fit_report(result_least_squares_bounds))
print("\nResults using Nelder-Mead optimization:")
print(fit_report(result_nelder))
print("\nResults using BFGS optimization:")
print(fit_report(result_bfgs))
print("\nResults using differential_evolution optimization:")
print(fit_report(result_differential_evolution))

print("\nResults using basinhopping optimization:")
print(fit_report(result_basinhopping))

# print("\nResults using brute optimization:")
# print(fit_report(result_brute))

# Create a list of dictionaries to store the results
results = []

# Add the results from each method to the list
results.append({
    'method': 'leastsq',
    'b': result_leastsq.params['b'].value,
    'omega': result_leastsq.params['omega'].value
})
results.append({
    'method': 'least_squares',
    'b': result_least_squares_bounds.params['b'].value,
    'omega': result_least_squares_bounds.params['omega'].value
})
results.append({
    'method': 'nelder',
    'b': result_nelder.params['b'].value,
    'omega': result_nelder.params['omega'].value
})
results.append({
    'method': 'bfgs',
    'b': result_bfgs.params['b'].value,
    'omega': result_bfgs.params['omega'].value
})
results.append({
    'method': 'differential_evolution',
    'b': result_differential_evolution.params['b'].value,
    'omega': result_differential_evolution.params['omega'].value
})

results.append({
    'method': 'basinhopping',
    'b': result_basinhopping.params['b'].value,
    'omega': result_basinhopping.params['omega'].value
})
# results.append({
#     'method': 'brute',
#     'b': result_brute.params['b'].value,
#     'omega': result_brute.params['omega'].value
# })

# Write the results to a CSV file
import csv

with open('results_{0}.csv'.format(n_mask), 'w', newline='') as csvfile:
    fieldnames = ['method', 'b', 'omega']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for result in results:
        writer.writerow(result)



Matlab ref omega:  1.752690409810814
Results using least sq:
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 101
    # data points      = 6000
    # variables        = 5
    chi-square         = 0.00732161
    reduced chi-square = 1.2213e-06
    Akaike info crit   = -81688.6380
    Bayesian info crit = -81655.1404
    R-squared          = 0.99430520
[[Variables]]
    a:      0.09263010 +/- 1.3195e-04 (0.14%) (init = 1)
    b:      0.18906751 +/- 3.6962e-04 (0.20%) (init = 1)
    omega:  1.75745294 +/- 3.6387e-04 (0.02%) (init = 1)
    phi:    1.82341922 +/- 0.00135037 (0.07%) (init = 1)
    c:      2.59106292 +/- 1.4335e-05 (0.00%) (init = 1)
[[Correlations]] (unreported correlations are < 0.100)
    C(a, b)       = +0.7207
    C(omega, phi) = -0.7098
    C(a, phi)     = +0.2051
    C(b, phi)     = +0.1485
    C(a, omega)   = -0.1470
    C(b, omega)   = -0.1059

Results using least squares with bounds:
[[Fit Statistics]]
    # fitting method   = least_squar

In [51]:
# # Create a list of dictionaries to store the results
# results = []

# # Add the results from each method to the list
# results.append({
#     'method': 'leastsq',
#     'b': result_leastsq.params['b'].value,
#     'omega': result_leastsq.params['omega'].value
# })
# results.append({
#     'method': 'least_squares',
#     'b': result_least_squares_bounds.params['b'].value,
#     'omega': result_least_squares_bounds.params['omega'].value
# })
# results.append({
#     'method': 'nelder',
#     'b': result_nelder.params['b'].value,
#     'omega': result_nelder.params['omega'].value
# })
# results.append({
#     'method': 'bfgs',
#     'b': result_bfgs.params['b'].value,
#     'omega': result_bfgs.params['omega'].value
# })
# results.append({
#     'method': 'differential_evolution',
#     'b': result_differential_evolution.params['b'].value,
#     'omega': result_differential_evolution.params['omega'].value
# })

# results.append({
#     'method': 'basinhopping',
#     'b': result_basinhopping.params['b'].value,
#     'omega': result_basinhopping.params['omega'].value
# })
# # results.append({
# #     'method': 'brute',
# #     'b': result_brute.params['b'].value,
# #     'omega': result_brute.params['omega'].value
# # })

# # Write the results to a CSV file
# import csv

# with open('results.csv', 'w', newline='') as csvfile:
#     fieldnames = ['method', 'b', 'omega']
#     writer = csv.DictWriter(csvfile, fieldnames=fieldnameQs)

#     writer.writeheader()
#     for result in results:
#         writer.writerow(result)


In [52]:
# import numpy as np
# import matplotlib.pyplot as plt
# from lmfit import Model, fit_report, minimize

# # Define the function to be fitted
# def func(x, a, b, omega, phi, c):
#     return a * np.exp(-b * x) * np.sin(omega * x + phi) + c

# # Define the callback function
# def callback(pars, iteration, progress, *args, **kws):
#     progress.append([p.value for p in pars.values()])
    
    

# # Load data from text files
# xdata = np.loadtxt('xdata.txt')
# ydata = np.loadtxt('data150_6.txt')

# # Define the model to be fitted
# model = Model(func)

# # Set initial parameter values and bounds
# params = model.make_params(a=0.1, b=0.5, omega=0.1, phi=0.1, c=1.0)
# params['a'].min = 0.0
# params['a'].max = 1.0
# params['b'].min = 0.0
# params['omega'].min = 0.0
# params['phi'].min = 0
# params['phi'].max = 2*np.pi

# # Define a list to store the progress of each algorithm
# progress_leastsq = []
# progress_least_squares_bounds = []
# progress_nelder = []
# progress_bfgs = []

# # Fit the model to the data with callback using different methods
# result_leastsq = model.fit(ydata, params, x=xdata, method='leastsq', iter_cb=callback, iter_cb_kwargs={'progress': progress_leastsq})
# result_least_squares_bounds = model.fit(ydata, params, x=xdata, method='leastsq', bounds=params, iter_cb=callback, iter_cb_kwargs={'progress': progress_least_squares_bounds})
# result_nelder = minimize(func=residual, params=params, args=(xdata, ydata), method='nelder', callback=callback, iter_cb_kwargs={'progress': progress_nelder})
# result_bfgs = minimize(func=residual, params=params, args=(xdata, ydata), method='bfgs', callback=callback, iter_cb_kwargs={'progress': progress_bfgs})

# # Plot the progress of each algorithm
# fig, axs = plt.subplots(2, 2)
# axs[0, 0].plot(progress_leastsq)
# axs[0, 0].set_title('leastsq')
# axs[0, 1].plot(progress_least_squares_bounds)
# axs[0, 1].set_title('leastsq with bounds')
# axs[1, 0].plot(progress_nelder)
# axs[1, 0].set_title('Nelder-Mead')
# axs[1, 1].plot(progress_bfgs)
# axs[1, 1].set_title('BFGS')
# plt.show()

# # Print the fit results for each method
# print("Results using least squares:")
# print(fit_report(result_leastsq))
# print("\nResults using least squares with bounds:")
# print(fit_report(result_least_squares_bounds))
# print("\nResults using Nelder-Mead optimization:")
# print(fit_report(result_nelder))
# print("\nResults using BFGS optimization:")
# print(fit_report(result_bfgs))
